In [2]:
import mlflow

# Step 2: Set up the MLflow tracking server
# mlflow.set_tracking_uri("http://ec2-3-81-236-116.compute-1.amazonaws.com:5000/")

# Step 2: Set up the MLflow tracking server - local
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df = pd.read_csv("dataset.csv").dropna(subset=["clean_comment"])
df.shape
#  Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df["category"] = df["category"].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=["category"])

ngram_range = (1, 3)  # Trigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_comment"], df["category"], test_size=0.2, random_state=42, stratify=df["category"]
)

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

adasyn = ADASYN(random_state=42)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)




In [3]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYSN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float("C", 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical("penalty", ["l1", "l2"])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver="liblinear", random_state=42)
    return accuracy_score(y_test, model.fit(X_train_resampled, y_train_resampled).predict(X_test_vec))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(
        C=best_params["C"], penalty=best_params["penalty"], solver="liblinear", random_state=42
    )

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)


# Run the experiment for Logistic Regression
run_optuna_experiment()

[I 2024-12-02 11:26:42,616] A new study created in memory with name: no-name-9cda0ba7-feb7-4a82-bfd6-e536344f0443
[I 2024-12-02 11:26:43,104] Trial 0 finished with value: 0.7752625119323605 and parameters: {'C': 0.61682993105501, 'penalty': 'l2'}. Best is trial 0 with value: 0.7752625119323605.
[I 2024-12-02 11:26:43,249] Trial 1 finished with value: 0.6376653484249284 and parameters: {'C': 0.0006967663094872085, 'penalty': 'l2'}. Best is trial 0 with value: 0.7752625119323605.
[I 2024-12-02 11:26:43,392] Trial 2 finished with value: 0.6211645983908359 and parameters: {'C': 0.00037076924530687617, 'penalty': 'l2'}. Best is trial 0 with value: 0.7752625119323605.
[I 2024-12-02 11:26:43,810] Trial 3 finished with value: 0.7880812764216555 and parameters: {'C': 3.544096726115603, 'penalty': 'l1'}. Best is trial 3 with value: 0.7880812764216555.
[I 2024-12-02 11:26:44,389] Trial 4 finished with value: 0.7779899086322105 and parameters: {'C': 1.4641724384925345, 'penalty': 'l2'}. Best is tr

🏃 View run LogisticRegression_ADASYSN_TFIDF_Trigrams at: http://127.0.0.1:5000/#/experiments/602507174562575349/runs/0d69bdbbc8a14872a25d3e189148f6c0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/602507174562575349
